# Dataset parser

This is notebook with code used to parse dataset of articles fo shpringer journals on comupter science. 
To do this it also uses the list of journals located in file 'data/journal_names.txt'

Link for the data: https://sn-scigraph.figshare.com/articles/dataset/Dataset_Articles/7376468

It contains functions what does:

1. Check if journals are present in the file
2. Parse the dataset to find the articles form the journals

In [1]:
import pandas as pd
import json 

In [3]:
# Loading journal list
journals = set()

for i in range(963):
    print('\r' + str(i), end = '')
    fl = open("articles/articles_" + str(i) + ".jsonl")
    for line in fl:
        data = json.loads(line)
        
        for el in data.get('isPartOf', []):
            if 'name' in el:
                journals.add(el['name'])

962

In [ ]:
def get_name(string):
    pos = string.find('.')
    return string[pos + 2:-1]

In [ ]:
# Cheking if journals exist
journal_names = []
with open('data/journal_names.txt') as fl:
    for el in fl:
        if not get_name(el) in journals:
            print(el)
        else:
            journal_names.append(get_name(el))

In [ ]:
# Loading articles
df = {'journal':[], 'date_published':[], 'name':[], 'keywords':[], 'url':[], 'abstract':[]}

for i in range(963):
    print('\r' + str(i), end = '')
    fl = open("articles/articles_" + str(i) + ".jsonl")
    for line in fl:
        data = json.loads(line)
        flag = True
        for el in data.get('isPartOf', []):
            if ('name' in el) and (el['name'] in journal_names) and flag:
                flag = False
                df['journal'].append(el['name'])
                df['name'].append(data.get('name', None))
                df['url'].append(data.get('url', None))
                df['date_published'].append(data.get('datePublished', None))
                df['keywords'].append(data.get('keywords', None))
                df['abstract'].append(data.get('description', None))
            

907

In [33]:
# Saving the dataset
df1 = pd.DataFrame(df)
df1.to_csv('data/data_table.csv')

In [ ]:
# Deleting further years from the table
df1 = pd.read_csv("data/data_table.csv", index_col=0)
df1['year_published'] = list(map(lambda x: int(x.split('-')[0]), df1['date_published']))
df = df1[df1['year_published'] >= 2003]
df = df.reset_index().drop(columns=['index'])
df.to_csv('data/res_table.csv')